# Salinity Setup:<br>Dask Random Search for NN Prediction of Sahelian Summer Rainfall
***

In [1]:
import numpy as np
import pandas as pd 
import xarray as xr


import matplotlib.pyplot as plt
%matplotlib inline

import scipy.stats as st

from sklearn.model_selection import ShuffleSplit, KFold
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import TensorBoard 
import tensorflow_addons as tfa

from tensorboard.plugins.hparams import api as hp
%load_ext tensorboard

import dask
from dask import delayed
import dask.bag as db

<br>

## 1. Dask Client
---

In [2]:
from dask.distributed import Client

client = Client(n_workers=1, threads_per_worker=4, memory_limit=64e9)
client

Client Scheduler: tcp://127.0.0.1:43933 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 4 Memory: 64.00 GB


<br>

## 2. Data Loading
***

**feat_pc:** Principal Components of climate indices. Used for model input.  
**labels:** from Sahelrainfall data serves as reference data.

In [3]:
features = xr.open_dataset('data/da_final_salinitymodel.nc').feat_pc.values
labels = xr.open_dataset('data/da_final_salinitymodel.nc').labels.values

<br>

## 3. MODEL SETUP AND TUNING
***

<br>

### Build Model Function
---

In [4]:
def BuildModel(HPARAMS):
    model = keras.Sequential([
#         layers.Dropout(0.1, input_shape=(20,)),
        layers.Dense(HPARAMS['n_units_l1'], 'relu' ,name="layer1", input_shape=(22,)),
        layers.Dropout(HPARAMS['dropout'], name='Dropout1'),
        
        layers.Dense(HPARAMS['n_units_l2'], 'relu', name="layer2"),
        layers.Dropout(HPARAMS['dropout'], name='Dropout2'),
        
        layers.Dense(HPARAMS['n_units_l3'], 'relu', name="layer3"),
        layers.Dropout(HPARAMS['dropout'], name='Dropout3'),
        
        layers.Dense(HPARAMS['n_units_l4'], 'relu', name="layer4"),
        layers.Dropout(HPARAMS['dropout'], name='Dropout4'),
        
        layers.Dense(HPARAMS['n_units_l5'], 'relu', name="layer5"),
        
        
        layers.Dense(1, name='output'), #activation='linear'
    ])
    model.compile(
        loss='mean_squared_error',
        optimizer=keras.optimizers.Adam(
            learning_rate=HPARAMS['learn_rate']
        )
    )
    return model

# testmodel = BuildModel({'optimizer': 'Adam', 'learn_rate': 0.1, 'n_units': 10, 'dropout':0.1})
# print(testmodel.summary())

<br>

### Single Run Training & Error Calculation Funktion
---

In [5]:
def TrainModel(SPLIT, HPARAM, features):
    
    model = BuildModel(HPARAM)
    
    hist_dir = parent_dir + f"/run-{HPARAM['grid_num']:04d}" + f"/split-{SPLIT['split_num']:02d}"
    
    train_history = model.fit(
        features[SPLIT['train_index']],
        labels[SPLIT['train_index']],
        batch_size=HPARAM['batch_size'],
        epochs=200,
        verbose=0,
        callbacks=[earlystop],
    )
    

    y_train = np.squeeze(model.predict(features[SPLIT['train_index']]))
    y_test = np.squeeze(model.predict(features[SPLIT['test_index']]))

    train_error = y_train - labels[SPLIT['train_index']]
    train_ae = np.absolute(train_error)
    train_mae = np.mean(train_ae)
    train_mad = np.median(np.absolute(train_error - np.median(train_error)))
    train_mse = np.mean(train_error**2)
    train_rmse = np.sqrt(train_mse)
    train_corr = st.pearsonr(y_train, labels[SPLIT['train_index']])[0]
    
    test_error = y_test - labels[SPLIT['test_index']]
    test_ae = np.absolute(test_error)
    test_mae = np.mean(test_ae)
    test_mad = np.median(np.absolute(test_error - np.median(test_error)))
    test_mse = np.mean(test_error**2)
    test_rmse = np.sqrt(test_mse)
    test_corr = st.pearsonr(y_test, labels[SPLIT['test_index']])[0]

    training_length = len(train_history.history['loss'])
    
    metrics = {
        'train_mae': train_mae,
        'train_mad': train_mad,
        'train_mse': train_mse,
        'train_rmse': train_rmse,
        'train_corr': train_corr,
        'test_mae': test_mae,
        'test_mad': test_mad,
        'test_mse': test_mse,
        'test_rmse': test_rmse,
        'test_corr': test_corr,
        'training_length': training_length,

    }
#     train_error = y_train - labels[SPLIT['train_index']]
#     train_mse = np.mean(train_error**2) 
#     train_corr = st.pearsonr(y_train, labels[SPLIT['train_index']])[0]
    
#     test_error = y_test - labels[SPLIT['test_index']]
#     test_mse = np.mean(test_error**2) 
#     test_corr = st.pearsonr(y_test, labels[SPLIT['test_index']])[0]


    
#     metrics = {
#         'train_mse': train_mse,
#         'train_corr': train_corr,
#         'test_mse': test_mse,
#         'test_corr': test_corr,
#         'training_length': training_length
#     }
    
    return metrics

<br>

### Cross-Validation and Log Function
---

In [6]:
def TuneModel(HPARAM):
    
    
    with tf.summary.create_file_writer(parent_dir + f"/run-{HPARAM['grid_num']:04d}").as_default():
        hp.hparams({
            HP_LEARN_RATE: HPARAM['learn_rate'],
            HP_NUMBER_HIDDEN_UNITS_L1: HPARAM['n_units_l1'],
            HP_NUMBER_HIDDEN_UNITS_L2: HPARAM['n_units_l2'],
            HP_NUMBER_HIDDEN_UNITS_L3: HPARAM['n_units_l3'],
            HP_NUMBER_HIDDEN_UNITS_L4: HPARAM['n_units_l4'],
            HP_NUMBER_HIDDEN_UNITS_L5: HPARAM['n_units_l5'],
            HP_BATCH_SIZE: HPARAM['batch_size'],
            HP_DROPOUT: HPARAM['dropout']
        })
                
        metrics = SPLITS.map(lambda SPLIT: TrainModel(SPLIT, HPARAM, features)).compute()

        train_mae = [metric['train_mae'] for metric in metrics]
        train_mad = [metric['train_mad'] for metric in metrics]
        train_mse = [metric['train_mse'] for metric in metrics]
        train_rmse = [metric['train_rmse'] for metric in metrics]
        train_corr = [metric['train_corr'] for metric in metrics]
        test_mae = [metric['test_mae'] for metric in metrics]
        test_mad = [metric['test_mad'] for metric in metrics]
        test_mse = [metric['test_mse'] for metric in metrics]
        test_rmse = [metric['test_rmse'] for metric in metrics]
        test_corr = [metric['test_corr'] for metric in metrics]
        training_length = [metric['training_length'] for metric in metrics]

#         train_mse = [metric['train_mse'] for metric in metrics]
#         train_corr = [metric['train_corr'] for metric in metrics]
#         test_mse = [metric['test_mse'] for metric in metrics]
#         test_corr = [metric['test_corr'] for metric in metrics]
        
        result = {
        'train_mae_mu': np.mean(train_mae),
        'train_mae_sig': np.std( train_mae),
        'train_mad_mu': np.mean(train_mad),
        'train_mad_sig': np.std( train_mad),
        'train_mse_mu': np.mean(train_mse),
        'train_mse_sig': np.std( train_mse),
        'train_rmse_mu': np.mean(train_rmse),
        'train_rmse_sig': np.std( train_rmse),
        'train_corr_mu': np.mean(train_corr),
        'train_corr_sig': np.std( train_corr),
        'test_mae_mu': np.mean(test_mae),
        'test_mae_sig': np.std( test_mae),
        'test_mad_mu': np.mean(test_mad),
        'test_mad_sig': np.std( test_mad),
        'test_mse_mu': np.mean(test_mse),
        'test_mse_sig': np.std( test_mse),
        'test_rmse_mu': np.mean(test_rmse),
        'test_rmse_sig': np.std( test_rmse),
        'test_corr_mu': np.mean(test_corr),
        'test_corr_sig': np.std( test_corr),
#         'train_mse_mu': np.mean(train_mse),
#         'train_mse_sig': np.std(train_mse),
#         'train_corr_mu': np.mean(train_corr),
#         'train_corr_sig': np.std(train_corr),
#         'test_mse_mu': np.mean(test_mse),
#         'test_mse_sig': np.std(test_mse),
#         'test_corr_mu': np.mean(test_corr),
#         'test_corr_sig': np.std(test_corr), 
        'training_length_mu': np.mean(training_length),
        'training_length_sig': np.std(training_length),             
        }

        tf.summary.scalar('train_mae_mu', result['train_mae_mu'], step=1)
        tf.summary.scalar('train_mae_sig', result['train_mae_sig'], step=1)
        tf.summary.scalar('train_mad_mu', result['train_mad_mu'], step=1)
        tf.summary.scalar('train_mad_sig', result['train_mad_sig'], step=1)
        tf.summary.scalar('train_mse_mu', result['train_mse_mu'], step=1)
        tf.summary.scalar('train_mse_sig', result['train_mse_sig'], step=1)
        tf.summary.scalar('train_rmse_mu', result['train_rmse_mu'],  step=1)
        tf.summary.scalar('train_rmse_sig', result['train_rmse_sig'], step=1)
        tf.summary.scalar('train_corr_mu', result['train_corr_mu'], step=1)
        tf.summary.scalar('train_corr_sig', result['train_corr_sig'], step=1)
        tf.summary.scalar('test_mae_mu', result['test_mae_mu'], step=1)
        tf.summary.scalar('test_mae_sig', result['test_mae_sig'], step=1)
        tf.summary.scalar('test_mad_mu', result['test_mad_mu'], step=1)
        tf.summary.scalar('test_mad_sig', result['test_mad_sig'], step=1)
        tf.summary.scalar('test_mse_mu', result['test_mse_mu'], step=1)
        tf.summary.scalar('test_mse_sig', result['test_mse_sig'], step=1)
        tf.summary.scalar('test_rmse_mu', result['test_rmse_mu'], step=1)
        tf.summary.scalar('test_rmse_sig', result['test_rmse_sig'], step=1)
        tf.summary.scalar('test_corr_mu', result['test_corr_mu'], step=1)
        tf.summary.scalar('test_corr_sig', result['test_corr_sig'], step=1)
        tf.summary.scalar('training_length_mu', result['training_length_mu'], step=1)
        tf.summary.scalar('training_length_sig', result['training_length_sig'], step=1)
        
#         tf.summary.scalar(METRIC_TRAIN_MSE_MU,        result['train_mse_mu'],  step=1)
#         tf.summary.scalar(METRIC_TRAIN_MSE_SIG,       result['train_mse_sig'],   step=1)
#         tf.summary.scalar(METRIC_TRAIN_CORR_MU,       result['train_corr_mu'], step=1)
#         tf.summary.scalar(METRIC_TRAIN_CORR_SIG,      result['train_corr_sig'],  step=1)
#         tf.summary.scalar(METRIC_TEST_MSE_MU,         result['test_mse_mu'],   step=1)
#         tf.summary.scalar(METRIC_TEST_MSE_SIG,        result['test_mse_sig'],    step=1)
#         tf.summary.scalar(METRIC_TEST_CORR_MU,        result['test_corr_mu'],  step=1)
#         tf.summary.scalar(METRIC_TEST_CORR_SIG,       result['test_corr_sig'],   step=1)
#         tf.summary.scalar(METRIC_TRAINING_LENGTH_MU,  result['training_length_mu'],  step=1)
#         tf.summary.scalar(METRIC_TRAINING_LENGTH_SIG, result['training_length_sig'],   step=1)
        
    return result

<br>

## 4. Setup and Start Random Search
---

<br>

### Set Parent Directory
---

In [7]:
parent_dir = 'logs/salinity_rand4_4/'

##### Clear Directory

In [8]:
# rm -rf logs/salinity_rand_1/*

<br>

### Hyperparameter Setup
***

In [9]:
###################################
#####EXAMPLE SETUP FOR TESTING#####
###################################

nruns = 200
e_s_patience = 5
units_opt = [5, 10, 20, 30, 40, 50, 60] #Discrete
learn_min = 0.001 #Log Scale
learn_max = 0.1 #Log Scale
batch_min = 3 #uniform
batch_max = 10 #uniform
drop_min = 0.1 #Uniform
drop_max = 0.5 #Uniform

#GRID SERACH HYPERPARAMETER#
#---------------------------
HP_LEARN_RATE = hp.HParam('learn_rate', hp.RealInterval(learn_min, learn_max),display_name='Learning Rate') # LogScale
HP_NUMBER_HIDDEN_UNITS_L1= hp.HParam('n_units_l1', hp.Discrete(units_opt),display_name='L1 Hidden Units') #uniform
HP_NUMBER_HIDDEN_UNITS_L2= hp.HParam('n_units_l2', hp.Discrete(units_opt),display_name='L2 Hidden Units') #uniform
HP_NUMBER_HIDDEN_UNITS_L3= hp.HParam('n_units_l3', hp.Discrete(units_opt),display_name='L3 Hidden Units') #uniform
HP_NUMBER_HIDDEN_UNITS_L4= hp.HParam('n_units_l4', hp.Discrete(units_opt),display_name='L4 Hidden Units') #uniform
HP_NUMBER_HIDDEN_UNITS_L5= hp.HParam('n_units_l5', hp.Discrete(units_opt),display_name='L5 Hidden Units') #uniform
HP_BATCH_SIZE = hp.HParam('batch_size', hp.IntInterval(batch_min, batch_max),display_name='Batch Size') #Uniform
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(drop_min, drop_max),display_name='Dropout') #Uniform



#CROSS VALIDATION PARAMETER (NO PART OF GRID SEARCH)#
#----------------------------------------------------
cv_param={
    'N_FOLDS': 10,         # number of folds -> small for Test Runs
}

#Early STOPPING (NO PART OF OPTIMISATION)#
#----------------------------------------#
earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=e_s_patience)

# hist_dir = parent_dir + f"/run-{HPARAM['grid_num']:04d}" + datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = keras.callbacks.TensorBoard(log_dir=hist_dir)

<br>

### Create HP Bag
***

In [10]:
learn_rate = 10**np.random.uniform(np.log10(learn_max),np.log10(learn_min), nruns)
n_units_l1 = np.random.choice(units_opt, nruns)
n_units_l2 = np.random.choice(units_opt, nruns)
n_units_l3 = np.random.choice(units_opt, nruns)
n_units_l4 = np.random.choice(units_opt, nruns)
n_units_l5 = np.random.choice(units_opt, nruns)
batch_size = np.random.uniform(batch_min, batch_max, nruns).astype(int)
dropout = np.random.uniform(drop_min, drop_max, nruns)

hparams = []

for i in range(nruns):

    hparams.append(
            {
            'grid_num': i,
            'learn_rate': learn_rate[i],
            'n_units_l1': n_units_l1[i],
            'n_units_l2': n_units_l2[i],
            'n_units_l3': n_units_l3[i],
            'n_units_l4': n_units_l4[i],
            'n_units_l5': n_units_l5[i],
            'batch_size': batch_size[i],
            'dropout': dropout[i],       
            }
        )
                        
HPARAMS = db.from_sequence(hparams, npartitions=10)

# HPARAMS.take(2,1)
# HPARAMS

<br>

### Create Data Splits Bag (KFold with permutation)
***

In [11]:
split_num = 0
splits = []
for train, test in KFold(n_splits=cv_param['N_FOLDS'], shuffle=True).split(features):                      #KFold

    train = shuffle(train)
    test  = shuffle(test)
# for train, test in ShuffleSplit(n_splits=cv_param['N_FOLDS'], test_size=cv_param['TEST_FRAC']).split(predictor_pc):
    splits.append(
        {
        'train_index': train,
        'test_index': test,
        'split_num': split_num
        }
    )
    split_num += 1 

SPLITS = db.from_sequence (splits, npartitions=1)

# SPLITS.take(1, 2)
# SPLITS

<br>

### Log Experiment Confiuration to TensorBoard
---

In [12]:
with tf.summary.create_file_writer(parent_dir).as_default():
    hp.hparams_config(
        hparams=[
            HP_LEARN_RATE,
            HP_NUMBER_HIDDEN_UNITS_L1,
            HP_NUMBER_HIDDEN_UNITS_L2,
            HP_NUMBER_HIDDEN_UNITS_L3,
            HP_NUMBER_HIDDEN_UNITS_L4,
            HP_NUMBER_HIDDEN_UNITS_L5,
            HP_BATCH_SIZE,
            HP_DROPOUT
        ],
        metrics=[
            hp.Metric('train_mae_mu',   display_name='Training Sample MAE µ'),
            hp.Metric('train_mae_sig',  display_name='Training Sample MAE σ'),
            hp.Metric('train_mad_mu',   display_name='Training Sample MAD µ'),
            hp.Metric('train_mad_sig',  display_name='Training Sample MAD σ'),
            hp.Metric('train_mse_mu',   display_name='Training Sample MSE µ'),
            hp.Metric('train_mse_sig',  display_name='Training Sample MSE σ'),
            hp.Metric('train_rmse_mu',   display_name='Training Sample RMSE µ'),
            hp.Metric('train_rmse_sig',  display_name='Training Sample RMSE σ'),
            hp.Metric('train_corr_mu',  display_name='Training Sample Correlation µ'),
            hp.Metric('train_corr_sig', display_name='Training Sample Correlation σ'),
            hp.Metric('test_mae_mu',   display_name='Test Sample MAE µ'),
            hp.Metric('test_mae_sig',  display_name='Test Sample MAE σ'),
            hp.Metric('test_mad_mu',   display_name='Test Sample MAD µ'),
            hp.Metric('test_mad_sig',  display_name='Test Sample MAD σ'),
            hp.Metric('test_mse_mu',   display_name='Test Sample MSE µ'),
            hp.Metric('test_mse_sig',  display_name='Test Sample MSE σ'),
            hp.Metric('test_rmse_mu',   display_name='Test Sample RMSE µ'),
            hp.Metric('test_rmse_sig',  display_name='Test Sample RMSE σ'),
            hp.Metric('test_corr_mu',  display_name='Test Sample Correlation µ'),
            hp.Metric('test_corr_sig', display_name='Test Sample Correlation σ'),            
            hp.Metric('training_length_mu', display_name='Training Length µ'),
            hp.Metric('training_length_sig', display_name='Training Length σ'),
        ],
    )

<br>

### Run Model
***

In [13]:
%%time
results = HPARAMS.map(lambda HPARAM: TuneModel(HPARAM)).compute()
client.close()

distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing


CPU times: user 18min 7s, sys: 33.7 s, total: 18min 41s
Wall time: 44min 12s


<br>
<br>

***
***
<br>
<br>